# Imports and Config

In [2]:
import os
from typing import Optional, Dict, Tuple
import numpy as np
import warnings

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
import random

from scipy.optimize import minimize_scalar
import pandas as pd

SEED = 99
NUM_RUNS = 50

os.makedirs("results", exist_ok=True)

# `AR1_Shock`

In [3]:
class AR1_Shock:
    def __init__(self, rho, sigma_eta, seed=None):
        self.rho = rho
        self.sigma_eta = sigma_eta
        self.rng = np.random.RandomState(seed)
        self.current = 0.0

    def reset(self):
        self.current = 0.0

    def generate_next(self):
        eta = self.rng.normal(0, self.sigma_eta)
        self.current = self.rho * self.current + eta
        return self.current


# `TheoreticalBenchmarks`

In [4]:
class TheoreticalBenchmarks:
    """Calculate theoretical Nash and Monopoly prices/profits for market models"""
    
    def __init__(self, seed=None):
        self.seed = seed
        if seed is not None:
            np.random.seed(seed)
    
    def calculate_logit_benchmarks(self, shock_cfg=None):
        """
        Calculate Nash and Monopoly benchmarks for Logit model
        With shocks: Uses Monte Carlo integration
        Without shocks: Uses analytical formulas
        """
        # Model parameters
        a = 2.0
        c = 1.0
        mu = 0.25
        a0 = 0.0
        
        if shock_cfg is None or not shock_cfg.get('enabled', False):
            # No shocks - use standard benchmarks
            p_N = 1.473
            p_M = 1.925
            
            # Calculate profits at these prices
            exp_N = np.exp((a - p_N) / mu)
            exp_outside = np.exp(a0 / mu)
            q_N = exp_N / (2 * exp_N + exp_outside)
            E_pi_N = (p_N - c) * q_N
            
            exp_M = np.exp((a - p_M) / mu)
            q_M = exp_M / (2 * exp_M + exp_outside)
            E_pi_M = (p_M - c) * q_M
            
            return {
                'p_N': p_N,
                'E_pi_N': E_pi_N,
                'p_M': p_M,
                'E_pi_M': E_pi_M,
                'shock_enabled': False
            }
        
        # With shocks - use Monte Carlo
        scheme_params = {
            'A': {'rho': 0.3, 'sigma_eta': 0.5},
            'B': {'rho': 0.95, 'sigma_eta': 0.05},
            'C': {'rho': 0.9, 'sigma_eta': 0.3}
        }
        
        scheme = shock_cfg.get('scheme', 'A')
        params = scheme_params[scheme.upper()]
        rho = params['rho']
        sigma_eta = params['sigma_eta']
        
        # Unconditional variance of shocks
        sigma2 = sigma_eta**2 / (1 - rho**2)
        sigma = np.sqrt(sigma2)
        
        # Monte Carlo samples
        N = 10000
        shock_mode = shock_cfg.get('mode', 'independent')
        
        if shock_mode == 'independent':
            xi_samples = np.random.normal(0, sigma, (N, 2))
        else:  # correlated
            xi_samples = np.random.normal(0, sigma, N)
        
        # Expected profit functions
        def E_pi1(p1, p2):
            if shock_mode == 'independent':
                exps1 = np.exp((a - p1 + xi_samples[:, 0]) / mu)
                exps2 = np.exp((a - p2 + xi_samples[:, 1]) / mu)
            else:
                exps1 = np.exp((a - p1 + xi_samples) / mu)
                exps2 = np.exp((a - p2 + xi_samples) / mu)
            den = exps1 + exps2 + np.exp(a0 / mu)
            qs = exps1 / den
            return np.mean((p1 - c) * qs)
        
        def E_pi2(p1, p2):
            if shock_mode == 'independent':
                exps1 = np.exp((a - p1 + xi_samples[:, 0]) / mu)
                exps2 = np.exp((a - p2 + xi_samples[:, 1]) / mu)
            else:
                exps1 = np.exp((a - p1 + xi_samples) / mu)
                exps2 = np.exp((a - p2 + xi_samples) / mu)
            den = exps1 + exps2 + np.exp(a0 / mu)
            qs = exps2 / den
            return np.mean((p2 - c) * qs)
        
        # Nash equilibrium
        def best_response(p_j):
            def neg_E_pi(p_i):
                return -E_pi1(p_i, p_j)
            res = minimize_scalar(neg_E_pi, bounds=(c, c + 5), method='bounded')
            return res.x
        
        p_guess = 1.5
        for _ in range(50):
            p_guess = best_response(p_guess)
        p_N = p_guess
        E_pi_N = E_pi1(p_N, p_N)
        
        # Monopoly
        def neg_E_joint(p):
            return -(E_pi1(p, p) + E_pi2(p, p))
        
        res_M = minimize_scalar(neg_E_joint, bounds=(c, c + 5), method='bounded')
        p_M = res_M.x
        E_pi_M = E_pi1(p_M, p_M)
        
        return {
            'p_N': p_N,
            'E_pi_N': E_pi_N,
            'p_M': p_M,
            'E_pi_M': E_pi_M,
            'shock_enabled': True,
            'scheme': scheme,
            'mode': shock_mode,
            'sigma': sigma
        }
    
    def calculate_hotelling_benchmarks(self, shock_cfg=None):
        """
        Calculate Nash and Monopoly benchmarks for Hotelling model
        Shocks don't affect expected benchmarks (linearity in net shock)
        """
        # Model parameters
        c = 0.0
        v_bar = 1.75
        theta = 1.0
        
        # Standard benchmarks (unchanged with shocks)
        p_N = 1.00
        p_M = 1.25
        
        # Calculate profits
        # At Nash: both firms charge 1, split market equally
        q_N = 0.5
        E_pi_N = p_N * q_N
        
        # At Monopoly: both charge 1.25, split market equally
        q_M = 0.5
        E_pi_M = p_M * q_M
        
        return {
            'p_N': p_N,
            'E_pi_N': E_pi_N,
            'p_M': p_M,
            'E_pi_M': E_pi_M,
            'shock_enabled': shock_cfg is not None and shock_cfg.get('enabled', False),
            'note': 'Shocks do not affect expected benchmarks for Hotelling'
        }
    
    def calculate_linear_benchmarks(self, shock_cfg=None):
        """
        Calculate Nash and Monopoly benchmarks for Linear model
        Shocks don't affect expected benchmarks (linearity in shocks)
        """
        # Model parameters
        c = 0.0
        a_bar = 1.0
        d = 0.25
        
        # Standard benchmarks (unchanged with shocks)
        p_N = 0.4286
        p_M = 0.5
        
        # Calculate profits
        denominator = 1 - d**2
        
        # At Nash
        q_N = (a_bar - p_N - d * (a_bar - p_N)) / denominator
        E_pi_N = p_N * q_N
        
        # At Monopoly
        q_M = a_bar - p_M
        E_pi_M = p_M * q_M
        
        return {
            'p_N': p_N,
            'E_pi_N': E_pi_N,
            'p_M': p_M,
            'E_pi_M': E_pi_M,
            'shock_enabled': shock_cfg is not None and shock_cfg.get('enabled', False),
            'note': 'Shocks do not affect expected benchmarks for Linear'
        }
    
    def calculate_all_benchmarks(self, shock_cfg=None):
        """Calculate benchmarks for all three models"""
        results = {
            'logit': self.calculate_logit_benchmarks(shock_cfg),
            'hotelling': self.calculate_hotelling_benchmarks(shock_cfg),
            'linear': self.calculate_linear_benchmarks(shock_cfg)
        }
        return results
    
    def generate_benchmark_table(self, shock_configs):
        """
        Generate a comprehensive table of benchmarks for different configurations
        
        Args:
            shock_configs: List of shock configurations to test
        
        Returns:
            pandas DataFrame with benchmarks
        """
        data = []
        
        for config in shock_configs:
            config_name = config.get('name', 'No Config')
            benchmarks = self.calculate_all_benchmarks(config)
            
            for model, bench in benchmarks.items():
                data.append({
                    'Configuration': config_name,
                    'Model': model.upper(),
                    'Nash Price': round(bench['p_N'], 4),
                    'Nash Profit': round(bench['E_pi_N'], 4),
                    'Monopoly Price': round(bench['p_M'], 4),
                    'Monopoly Profit': round(bench['E_pi_M'], 4),
                    'Shock Enabled': bench['shock_enabled']
                })
        
        return pd.DataFrame(data)

# `MarketEnv` Class

In [5]:
class MarketEnv:
    """Base Market environment supporting all three models with optional shocks"""
   
    def __init__(
        self,
        market_model: str = "logit",
        shock_cfg: Optional[Dict] = None,
        n_firms: int = 2,
        horizon: int = 10000,
        seed: Optional[int] = None
    ):
        self.model = market_model.lower()
        self.n_firms = n_firms
        self.N = 15 # Number of discrete prices
        self.horizon = horizon
        self.t = 0
        self.rng = np.random.RandomState(seed)
       
        # Model-specific parameters (verified against PDFs)
        if self.model == "logit":
            self.c = 1.0
            self.a = 2.0
            self.a0 = 0.0
            self.mu = 0.25
            self.P_N = 1.473
            self.P_M = 1.925
        elif self.model == "hotelling":
            self.c = 0.0
            self.v_bar = 1.75
            self.theta = 1.0
            self.P_N = 1.00
            self.P_M = 1.25
        elif self.model == "linear":
            self.c = 0.0
            self.a_bar = 1.0
            self.d = 0.25
            
            # Nash (Cournot duopoly)
            self.P_N = self.a_bar * (1 - self.d) / (2 - self.d)  # 0.4286
            q_N = self.P_N  # By symmetry at equilibrium
            pi_N = self.P_N * q_N  # 0.1959
            
            # Monopoly (single firm, no competition)
            self.P_M = self.a_bar / 2  # 0.5
            q_M = self.a_bar - self.P_M  # 0.5 (residual demand, NOT Cournot!)
            pi_M = self.P_M * q_M  # 0.25
            
            # Store for Delta calculations
            self.pi_N_linear = pi_N  # 0.1959
            self.pi_M_linear = pi_M  # 0.25
            
            # Verify profit range is healthy
            profit_range = pi_M - pi_N  # Should be ~0.05
            if profit_range < 0.04:
                import warnings
                warnings.warn(f"Linear model profit range too small: {profit_range:.4f}")
        else:
            raise ValueError(f"Unknown model: {self.model}")
       
        # Construct price grid
        span = self.P_M - self.P_N
        self.price_grid = np.linspace(
            self.P_N - 0.15 * span,
            self.P_M + 0.15 * span,
            self.N
        )
       
        # Initialize shock configuration
        self.shock_cfg = shock_cfg or {}
        self.shock_enabled = self.shock_cfg.get("enabled", False)
       
        if self.shock_enabled:
            self.shock_mode = self.shock_cfg.get("mode", "correlated")
            scheme = self.shock_cfg.get("scheme", None)
           
            # Get AR(1) parameters (verified against PDFs)
            if scheme:
                scheme_params = {
                    'A': {'rho': 0.3, 'sigma_eta': 0.5},
                    'B': {'rho': 0.95, 'sigma_eta': 0.05},
                    'C': {'rho': 0.9, 'sigma_eta': 0.3}
                }
                if scheme.upper() in scheme_params:
                    params = scheme_params[scheme.upper()]
                    self.rho = params['rho']
                    self.sigma_eta = params['sigma_eta']
                else:
                    raise ValueError(f"Unknown scheme: {scheme}")
            else:
                self.rho = self.shock_cfg.get('rho', 0.9)
                self.sigma_eta = self.shock_cfg.get('sigma_eta', 0.15)
           
            # Initialize shock generators
            if self.shock_mode == "independent":
                self.shock_generators = [
                    AR1_Shock(self.rho, self.sigma_eta, seed=seed+i if seed else None)
                    for i in range(self.n_firms)
                ]
            else: # correlated
                self.shock_generator = AR1_Shock(self.rho, self.sigma_eta, seed=seed)
           
            self.current_shocks = np.zeros(self.n_firms)
       
        self.reset()
   
    def reset(self):
        """Reset environment - returns state as actual prices"""
        self.t = 0
       
        # Reset shocks
        if self.shock_enabled:
            if self.shock_mode == "independent":
                for gen in self.shock_generators:
                    gen.reset()
            else:
                self.shock_generator.reset()
            self.current_shocks = np.zeros(self.n_firms)
       
        # Initialize prices at middle of grid
        mid_idx = self.N // 2
        self.current_price_idx = np.array([mid_idx] * self.n_firms)
       
        return self._get_state()
   
    def _get_state(self):
        """Get current state as actual prices (numpy array)"""
        return self.price_grid[self.current_price_idx].copy()
    
    def _get_state_indices(self):
        """Get current state as price indices (for Q-learning compatibility)"""
        return tuple(self.current_price_idx)
   
    def _generate_shocks(self):
        """Generate next period shocks"""
        if not self.shock_enabled:
            return np.zeros(self.n_firms)
       
        if self.shock_mode == "independent":
            shocks = np.array([gen.generate_next() for gen in self.shock_generators])
        else:
            shock = self.shock_generator.generate_next()
            shocks = np.array([shock] * self.n_firms)
       
        return shocks
   
    def get_current_shocks(self):
        """Get current shocks (for PSO evaluation)"""
        return self.current_shocks.copy() if self.shock_enabled else np.zeros(self.n_firms)
   
    def calculate_demand_and_profit(self, prices: np.ndarray, shocks: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Calculate demand and profit given prices and shocks"""
        if self.model == "logit":
            return self._logit_demand_profit(prices, shocks)
        elif self.model == "hotelling":
            return self._hotelling_demand_profit(prices, shocks)
        elif self.model == "linear":
            return self._linear_demand_profit(prices, shocks)
   
    def _logit_demand_profit(self, prices: np.ndarray, shocks: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Logit demand - shocks affect variance but not Nash equilibrium
        
        Critical: E[demand | shocks] = demand(no shocks) since E[ε] = 0
        Nash price must remain constant regardless of shock scheme
        """
        # Base utilities WITHOUT shocks (defines Nash equilibrium)
        base_utilities = (self.a - prices) / self.mu
        
        # Add shocks OUTSIDE mu scaling to preserve E[utility]
        realized_utilities = base_utilities + shocks
        
        # Numerical stability
        max_util = np.max(realized_utilities)
        exp_utils = np.exp(realized_utilities - max_util)
        exp_outside = np.exp(self.a0/self.mu - max_util)
        
        denominator = np.sum(exp_utils) + exp_outside
        demands = exp_utils / denominator
        profits = (prices - self.c) * demands
        
        return demands, profits
   
    def _hotelling_demand_profit(self, prices: np.ndarray, shocks: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Hotelling demand with net shock affecting boundary (verified against PDF)"""
        p1, p2 = prices[0], prices[1]
        epsilon_net = shocks[0] - shocks[1] if self.n_firms == 2 else 0
       
        x_hat = 0.5 + (p2 - p1 + epsilon_net) / (2 * self.theta)
        x_hat = np.clip(x_hat, 0, 1)
       
        q1 = x_hat
        q2 = 1 - x_hat
        demands = np.array([q1, q2])
        profits = prices * demands
       
        return demands, profits
   
    def _linear_demand_profit(self, prices: np.ndarray, shocks: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Linear demand with additive shocks (verified against PDF)"""
        a_shocked = self.a_bar + shocks
        denominator = 1 - self.d**2
       
        q1 = ((a_shocked[0] - prices[0]) - self.d * (a_shocked[1] - prices[1])) / denominator
        q2 = ((a_shocked[1] - prices[1]) - self.d * (a_shocked[0] - prices[0])) / denominator
       
        q1 = max(0, q1)
        q2 = max(0, q2)
        demands = np.array([q1, q2])
        profits = prices * demands
       
        return demands, profits
   
    def step(self, action_indices):
        """Execute one step - returns (state_prices, profits, done, info)
        
        Args:
            action_indices: Array of price indices for each agent
            
        Returns:
            next_state: Numpy array of actual prices (not indices)
            profits: Numpy array of profits for each agent
            done: Episode termination flag
            info: Dictionary with prices, demands, and shocks
        """
        action_indices = np.asarray(action_indices, dtype=int)
       
        # Update price indices
        self.current_price_idx = action_indices
        prices = self.price_grid[self.current_price_idx]
       
        # Generate shocks for this period
        self.current_shocks = self._generate_shocks()
       
        # Calculate demands and profits
        demands, profits = self.calculate_demand_and_profit(prices, self.current_shocks)
       
        # Update time
        self.t += 1
       
        # Get next state (actual prices, not indices)
        next_state = self._get_state()
       
        # Episode termination
        done = False
       
        # Info dictionary
        info = {
            'prices': prices.copy(),
            'demands': demands.copy(),
            'shocks': self.current_shocks.copy(),
            'price_indices': self.current_price_idx.copy()  # Include indices for Q-learning
        }
       
        return next_state, profits, done, info

class MarketEnvContinuous(MarketEnv):
    """Market environment that accepts both discrete indices and continuous prices as actions"""
    
    def step(self, actions):
        """Execute one step with flexible action handling
        
        Args:
            actions: List/array where each element can be:
                     - int/np.integer: discrete price index
                     - float: continuous price value
                     
        Returns:
            next_state: Numpy array of actual prices
            profits: Numpy array of profits
            done: Episode termination flag
            info: Dictionary with prices, demands, shocks, and indices
        """
        prices = []
        indices = []
        
        for a in actions:
            if isinstance(a, (int, np.integer)):
                # Discrete action: use price from grid
                prices.append(self.price_grid[a])
                indices.append(a)
            else:
                # Continuous action: use actual price value
                price = float(a)
                prices.append(price)
                # Find closest grid index for tracking
                indices.append(np.argmin(np.abs(self.price_grid - price)))
        
        prices = np.array(prices)
        self.current_price_idx = np.array(indices)
        
        # Generate shocks
        self.current_shocks = self._generate_shocks()
        
        # Calculate demands and profits
        demands, profits = self.calculate_demand_and_profit(prices, self.current_shocks)
        
        # Update time
        self.t += 1
        
        # Get next state (actual prices)
        next_state = self._get_state()
        
        # Episode termination
        done = False
        
        # Info dictionary
        info = {
            'prices': prices.copy(),
            'demands': demands.copy(),
            'shocks': self.current_shocks.copy(),
            'price_indices': self.current_price_idx.copy()
        }
        
        return next_state, profits, done, info

    def get_state_prices(self):
        """Get current state as actual prices (same as _get_state)"""
        return self._get_state()


# `QLearningAgent`

In [6]:
class QLearningAgent:
    def __init__(self, n_actions, alpha=0.15, gamma=0.95, beta=1.5e-4, agent_id=0, price_grid=None):
        """
        Q-Learning Agent for discrete pricing decisions.
        
        Args:
            n_actions: Number of discrete price points
            alpha: Learning rate
            gamma: Discount factor
            beta: Exploration decay rate
            agent_id: Agent identifier (0 or 1)
            price_grid: Price grid for converting continuous states to indices (optional)
        """
        self.Q = np.zeros((n_actions, n_actions))
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.beta = beta  # Exploration decay rate
        self.t = 0
        self.id = agent_id
        self.n_actions = n_actions
        self.price_grid = price_grid  # Store price grid for state conversion
    
    def _state_to_indices(self, state):
        """Convert continuous state (prices) to discrete indices
        
        Args:
            state: Can be either:
                   - tuple of indices (old format, backward compatible)
                   - numpy array of prices (new format)
                   
        Returns:
            tuple of indices
        """
        if isinstance(state, tuple):
            # Already indices (backward compatible)
            return state
        elif isinstance(state, np.ndarray):
            # Convert prices to indices
            if self.price_grid is not None:
                indices = tuple(np.argmin(np.abs(self.price_grid - price)) for price in state)
                return indices
            else:
                # Fallback: assume state is small integers that can be indices
                return tuple(state.astype(int))
        else:
            raise ValueError(f"Unknown state format: {type(state)}")

    def choose_action(self, state):
        """Choose action using epsilon-greedy with exponential decay
        
        Args:
            state: Either tuple of indices or numpy array of prices
            
        Returns:
            action: Discrete action index
        """
        state_idx = self._state_to_indices(state)
        opp_idx = state_idx[1 - self.id]
        
        epsilon = np.exp(-self.beta * self.t)
        if np.random.rand() < epsilon:
            return np.random.randint(self.n_actions)  # Explore
        else:
            return np.argmax(self.Q[opp_idx])  # Exploit

    def update(self, state, action, reward, next_state):
        """Update Q-table using TD learning
        
        Args:
            state: Current state (indices or prices)
            action: Action taken
            reward: Reward received
            next_state: Next state (indices or prices)
        """
        state_idx = self._state_to_indices(state)
        next_state_idx = self._state_to_indices(next_state)
        
        opp_idx = state_idx[1 - self.id]
        next_opp_idx = next_state_idx[1 - self.id]
        
        td_target = reward + self.gamma * np.max(self.Q[next_opp_idx])
        self.Q[opp_idx, action] += self.alpha * (td_target - self.Q[opp_idx, action])
        self.t += 1


# `DDPGAgent`

In [7]:
class ReplayBuffer:
    """Experience replay buffer for DDPG"""
    def __init__(self, capacity=100000):
        self.buffer = deque(maxlen=capacity)
   
    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))
   
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        return (np.array(states), np.array(actions), np.array(rewards),
                np.array(next_states), np.array(dones))
   
    def __len__(self):
        return len(self.buffer)

class Actor(nn.Module):
    """Actor network for DDPG"""
    def __init__(self, state_dim, action_dim, hidden_dim=400):
        super(Actor, self).__init__()
        self.bn_input = nn.BatchNorm1d(state_dim)
        self.fc1 = nn.Linear(state_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 300)
        self.bn2 = nn.BatchNorm1d(300)
        self.fc3 = nn.Linear(300, action_dim)
       
        # Initialize weights
        self._init_weights()
   
    def _init_weights(self):
        nn.init.uniform_(self.fc1.weight, -1/np.sqrt(self.fc1.in_features), 1/np.sqrt(self.fc1.in_features))
        nn.init.uniform_(self.fc2.weight, -1/np.sqrt(self.fc2.in_features), 1/np.sqrt(self.fc2.in_features))
        nn.init.uniform_(self.fc3.weight, -3e-3, 3e-3)
   
    def forward(self, state):
        x = self.bn_input(state)
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        x = torch.tanh(self.fc3(x))
        return x

class Critic(nn.Module):
    """Critic network for DDPG"""
    def __init__(self, state_dim, action_dim, hidden_dim=400):
        super(Critic, self).__init__()
        self.bn_input = nn.BatchNorm1d(state_dim)
        self.fc1 = nn.Linear(state_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.fc2 = nn.Linear(hidden_dim + action_dim, 300)
        self.fc3 = nn.Linear(300, 1)
       
        # Initialize weights
        self._init_weights()
   
    def _init_weights(self):
        nn.init.uniform_(self.fc1.weight, -1/np.sqrt(self.fc1.in_features), 1/np.sqrt(self.fc1.in_features))
        nn.init.uniform_(self.fc2.weight, -1/np.sqrt(self.fc2.in_features), 1/np.sqrt(self.fc2.in_features))
        nn.init.uniform_(self.fc3.weight, -3e-3, 3e-3)
   
    def forward(self, state, action):
        x = self.bn_input(state)
        x = F.relu(self.bn1(self.fc1(x)))
        x = torch.cat([x, action], dim=1)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class OUNoise:
    """Ornstein-Uhlenbeck process for exploration"""
    def __init__(self, action_dim, mu=0.0, theta=0.15, sigma=0.2):
        self.action_dim = action_dim
        self.mu = mu
        self.theta = theta
        self.sigma = sigma
        self.state = np.ones(self.action_dim) * self.mu
        self.reset()
   
    def reset(self):
        self.state = np.ones(self.action_dim) * self.mu
   
    def sample(self):
        dx = self.theta * (self.mu - self.state)
        dx += self.sigma * np.random.randn(self.action_dim)
        self.state += dx
        return self.state.copy()

class DDPGAgent:
    """Deep Deterministic Policy Gradient Agent for pricing competition simulation"""
    def __init__(
        self,
        agent_id,
        state_dim,
        action_dim,
        hidden_dim=400,
        actor_lr=1e-4,
        critic_lr=1e-3,
        gamma=0.99,
        tau=0.001,
        buffer_size=1000000,
        batch_size=64,
        seed=None,
        price_min=0.0,
        price_max=2.0
    ):
        self.agent_id = agent_id
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.gamma = gamma
        self.tau = tau
        self.batch_size = batch_size
        self.price_min = price_min
        self.price_max = price_max
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        if seed is not None:
            torch.manual_seed(seed)
            if self.device.type == 'cuda':
                torch.cuda.manual_seed(seed)
            np.random.seed(seed)
            random.seed(seed)
        
        # For deterministic behavior on GPU
        if self.device.type == 'cuda':
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
        
        # Actor networks
        self.actor = Actor(state_dim, action_dim, hidden_dim).to(self.device)
        self.actor_target = Actor(state_dim, action_dim, hidden_dim).to(self.device)
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=actor_lr)
        
        # Critic networks
        self.critic = Critic(state_dim, action_dim, hidden_dim).to(self.device)
        self.critic_target = Critic(state_dim, action_dim, hidden_dim).to(self.device)
        self.critic_target.load_state_dict(self.critic.state_dict())
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=critic_lr, weight_decay=1e-2)
        
        # Replay buffer
        self.replay_buffer = ReplayBuffer(buffer_size)
        
        # Noise process
        self.noise = OUNoise(action_dim)
        
        # Exploration parameters
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
   
    def select_action(self, state, explore=True):
        """Select action using actor network with optional exploration noise"""
        state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        self.actor.eval()
        with torch.no_grad():
            action = self.actor(state).cpu().numpy()[0]
        self.actor.train()
        
        if explore:
            noise = self.noise.sample() * self.epsilon
            action += noise
            action = np.clip(action, -1, 1)
        
        normalized_action = action.copy()  # For remember/replay (normalized [-1,1])
        
        # Scale to [price_min, price_max]
        scaled_price = self.price_min + (self.price_max - self.price_min) * (action[0] + 1) / 2
        
        return scaled_price, normalized_action
   
    def remember(self, state, action, reward, next_state, done):
        """Store experience in replay buffer"""
        self.replay_buffer.push(state, action, reward, next_state, done)
   
    def replay(self):
        """Train on a batch of experiences from replay buffer"""
        if len(self.replay_buffer) < self.batch_size:
            return
       
        # Sample batch
        states, actions, rewards, next_states, dones = self.replay_buffer.sample(self.batch_size)
       
        states = torch.FloatTensor(states).to(self.device)
        actions = torch.FloatTensor(actions).to(self.device)
        rewards = torch.FloatTensor(rewards).unsqueeze(1).to(self.device)
        next_states = torch.FloatTensor(next_states).to(self.device)
        dones = torch.FloatTensor(dones).unsqueeze(1).to(self.device)
       
        # Update critic
        with torch.no_grad():
            next_actions = self.actor_target(next_states)
            target_q = self.critic_target(next_states, next_actions)
            target_q = rewards + (1 - dones) * self.gamma * target_q
       
        current_q = self.critic(states, actions)
        critic_loss = F.mse_loss(current_q, target_q)
       
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()
       
        # Update actor
        actor_loss = -self.critic(states, self.actor(states)).mean()
       
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
       
        # Soft update target networks
        self._soft_update(self.actor, self.actor_target)
        self._soft_update(self.critic, self.critic_target)
   
    def _soft_update(self, local_model, target_model):
        """Soft update target network parameters: θ_target = τ*θ_local + (1 - τ)*θ_target"""
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(self.tau * local_param.data + (1.0 - self.tau) * target_param.data)
   
    def update_epsilon(self):
        """Decay exploration rate"""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
   
    def reset_noise(self):
        """Reset the noise process"""
        self.noise.reset()


# Simulation

In [8]:
def run_simulation(model, seed, shock_cfg, benchmarks):
    """Run Q-Learning vs DDPG simulation"""
    np.random.seed(seed)
    
    env = MarketEnvContinuous(market_model=model, shock_cfg=shock_cfg, seed=seed)
    
    price_min = env.price_grid.min()
    price_max = env.price_grid.max()
    
    q_agent = QLearningAgent(env.N, agent_id=0, price_grid=env.price_grid)
    ddpg_agent = DDPGAgent(
        agent_id=1,
        state_dim=2,
        action_dim=1,
        seed=seed,
        price_min=price_min,
        price_max=price_max
    )
    
    state = env.reset()
    profits_history = []
    prices_history = []
    
    for t in range(env.horizon):
        # Q-Learning selects discrete action index
        q_action_idx = q_agent.choose_action(state)
        q_price = env.price_grid[q_action_idx]
        
        # DDPG selects continuous price
        ddpg_state = state.astype(np.float32)
        ddpg_price, ddpg_norm = ddpg_agent.select_action(ddpg_state, explore=True)
        
        actions = [q_price, ddpg_price]
        next_state, rewards, done, info = env.step(actions)
        
        # Update Q-Learning
        q_agent.update(state, q_action_idx, rewards[0], next_state)
        
        # Update DDPG
        next_ddpg_state = next_state.astype(np.float32)
        ddpg_agent.remember(ddpg_state, ddpg_norm, rewards[1], next_ddpg_state, done)
        ddpg_agent.replay()
        
        if t % 100 == 0:
            ddpg_agent.update_epsilon()
        
        state = next_state
        prices_history.append(info['prices'])
        profits_history.append(rewards)
    
    # Calculate averages over last 1000 steps
    last_prices = np.array(prices_history[-1000:])
    avg_price_q = np.mean(last_prices[:, 0])
    avg_price_ddpg = np.mean(last_prices[:, 1])
    
    last_profits = np.array(profits_history[-1000:])
    avg_profit_q = np.mean(last_profits[:, 0])
    avg_profit_ddpg = np.mean(last_profits[:, 1])
    
    # Get benchmarks
    pi_n = benchmarks['E_pi_N']
    pi_m = benchmarks['E_pi_M']
    p_n = benchmarks['p_N']
    p_m = benchmarks['p_M']
    
    # Calculate Delta (profit-based)
    delta_q = (avg_profit_q - pi_n) / (pi_m - pi_n) if (pi_m - pi_n) != 0 else 0
    delta_ddpg = (avg_profit_ddpg - pi_n) / (pi_m - pi_n) if (pi_m - pi_n) != 0 else 0
    
    # Calculate RPDI (pricing-based)
    rpdi_q = (avg_price_q - p_n) / (p_m - p_n) if (p_m - p_n) != 0 else 0
    rpdi_ddpg = (avg_price_ddpg - p_n) / (p_m - p_n) if (p_m - p_n) != 0 else 0
    
    return avg_price_q, avg_price_ddpg, delta_q, delta_ddpg, rpdi_q, rpdi_ddpg, p_n


def main():
    shock_cfg = {
        'enabled': True,
        'scheme': 'B',
        'mode': 'independent'
    }
    
    benchmark_calculator = TheoreticalBenchmarks(seed=SEED)
    
    print("=" * 80)
    print("Q-LEARNING vs DDPG - SCHEME B")
    print("=" * 80)
    
    all_benchmarks = benchmark_calculator.calculate_all_benchmarks(shock_cfg)
    
    models = ['logit', 'hotelling', 'linear']
    results = {}
    
    for model in models:
        print(f"\nRunning {model.upper()} simulations...")
        
        model_benchmarks = all_benchmarks[model]
        
        avg_prices_q = []
        avg_prices_ddpg = []
        deltas_q = []
        deltas_ddpg = []
        rpdis_q = []
        rpdis_ddpg = []
        theo_prices = []
        
        for run in range(NUM_RUNS):
            seed = SEED + run
            apq, apd, dq, dd, rq, rd, p_n = run_simulation(model, seed, shock_cfg, model_benchmarks)
            avg_prices_q.append(apq)
            avg_prices_ddpg.append(apd)
            deltas_q.append(dq)
            deltas_ddpg.append(dd)
            rpdis_q.append(rq)
            rpdis_ddpg.append(rd)
            theo_prices.append(p_n)
        
        results[model] = {
            'Avg Price Q': np.mean(avg_prices_q),
            'Theo Price': np.mean(theo_prices),
            'Avg Price DDPG': np.mean(avg_prices_ddpg),
            'Delta Q': np.mean(deltas_q),
            'Delta DDPG': np.mean(deltas_ddpg),
            'RPDI Q': np.mean(rpdis_q),
            'RPDI DDPG': np.mean(rpdis_ddpg)
        }
        
        print(f"  Completed: Q Δ = {results[model]['Delta Q']:.3f}, DDPG Δ = {results[model]['Delta DDPG']:.3f}")
    
    data = {
        'Model': [m.upper() for m in models],
        'Q Avg. Prices': [round(results[m]['Avg Price Q'], 2) for m in models],
        'Theo. Prices': [round(results[m]['Theo Price'], 2) for m in models],
        'DDPG Avg. Prices': [round(results[m]['Avg Price DDPG'], 2) for m in models],
        'Theo. Prices ': [round(results[m]['Theo Price'], 2) for m in models],
        'Q Extra-profits Δ': [round(results[m]['Delta Q'], 2) for m in models],
        'DDPG Extra-profits Δ': [round(results[m]['Delta DDPG'], 2) for m in models],
        'Q RPDI': [round(results[m]['RPDI Q'], 2) for m in models],
        'DDPG RPDI': [round(results[m]['RPDI DDPG'], 2) for m in models]
    }
    
    df = pd.DataFrame(data)
    df.to_csv("./results/q_vs_ddpg.csv", index=False)
    
    print("\n" + "=" * 80)
    print("FINAL RESULTS")
    print("=" * 80)
    print(df.to_string(index=False))
    
    # Overall averages
    print("\n" + "=" * 80)
    print("OVERALL AVERAGES ACROSS ALL MODELS")
    print("=" * 80)
    
    avg_delta_q = np.mean([results[m]['Delta Q'] for m in models])
    avg_delta_ddpg = np.mean([results[m]['Delta DDPG'] for m in models])
    avg_rpdi_q = np.mean([results[m]['RPDI Q'] for m in models])
    avg_rpdi_ddpg = np.mean([results[m]['RPDI DDPG'] for m in models])
    
    print("\nQ-Learning:")
    print(f"  Average Delta (Δ): {avg_delta_q:.4f}")
    print(f"  Average RPDI:      {avg_rpdi_q:.4f}")
    print("\nDDPG:")
    print(f"  Average Delta (Δ): {avg_delta_ddpg:.4f}")
    print(f"  Average RPDI:      {avg_rpdi_ddpg:.4f}")
    
    print("\n[Results saved to ./results/q_vs_ddpg.csv]")


if __name__ == "__main__":
    main()

Q-LEARNING vs DDPG - SCHEME B

Running LOGIT simulations...


KeyboardInterrupt: 